In [1]:
# -*- coding:utf-8 -*-
from math import pi

import numpy as np
import pandas as pd

# Load data from Neighbourhoods & Supply Stations
neighborhoodsTable = pd.read_csv('./datasets/FiveNeighbourhoods.csv')
stationTable = pd.read_csv('./datasets/TwoSupplyStation.csv')


# Coordinate map includes both longtitude and latitude
coordMap = {}

for idx in range(len(neighborhoodsTable)):
  coordMap[neighborhoodsTable.iloc[idx,:]['Name']] = (neighborhoodsTable.iloc[idx,:]['Longitude'],
                                                      neighborhoodsTable.iloc[idx,:]['Latitude'])
for idx in range(len(stationTable)):
  coordMap[stationTable.iloc[idx,:]['Name']] = (stationTable.iloc[idx,:]['Longitude'],
                                                stationTable.iloc[idx,:]['Latitude'])

# Calculate the linear distance between two nodes twice according to latitude and longitude
def getDistance(coordA, coordB):

  # Angle Coordinates are converted to Radian Coordinates
  lat1 = coordA[0] * pi/180
  lat2 = coordB[0] * pi/180
  lng1 = coordA[1] * pi/180
  lng2 = coordB[1] * pi/180

  # Compute the intersection angle
  ang = np.cos(lat2) * np.cos(lat1) * np.cos(lng2-lng1) + np.sin(lat1) * np.sin(lat2)

  # Calculate twice the arc length of two nodes
  distance = np.arccos(ang) * 6371.004 * 2
  return distance

# Calculate how many intermediate nodes are required between two places
def getMidNodeNum(coordA, coordB):
  
  vehicleSpeed = 10         # default vehicle speed is 10km/h

  distance = getDistance(coordA, coordB)
  # if distance is smaller than the vehicle speed then task can be done within one hour
  # thus, return -1 indicates no intermediate node is required
  if distance < vehicleSpeed:
    return -1
  else:

    # index
    idx = distance // vehicleSpeed
    
    # Rounding method: 
    # 1) One middle point is required: takes two hours to move 20 kilometers
    # 2) Two middle points are required: takes three hours to move 26 kilometers
    if distance % vehicleSpeed >= 5:
      return int(idx)
    else:
      return int(idx-1)

Z = {}           
Mid = []         # to store the name of middle points
hasMidNode = []  # to record whether an intermediate node has been computed


# use for loop to traverse each neighbourhood and supply station
for i in coordMap.keys():
  for j in coordMap.keys():
    
    # A road is connected if the start and end points are together，then z[i,j] = z[j,i] = 1
    if i == j:
      Z[(i,j)] = 1
      Z[(j,i)] = 1
    else:
      midNodeNum = getMidNodeNum(coordMap[i], coordMap[j])
      
      # if getMidNodeNum() return no intermediate node，
      # then the two places can be directly connected，
      # which means z[i,j] = z[j,i] = 1
      if midNodeNum == -1 or midNodeNum == 0:
        Z[(i,j)] = 1
        Z[(j,i)] = 1
        
      else:
        # if the start and end points are not together and there are intermediate points,
        # then take actions below:
        # The naming format of intermediate points is "start-end-number"
        # If the two routes from a to b or b to a are not counted in the variable hasMidNode, it is a new point.
        # (The two routes are essentially the same, because the intermediate nodes between the two points are the same.
        # In order to reduce the amount of data, this value is calculated once)
        
        if i+"-"+j not in hasMidNode and j+"-"+i not in hasMidNode:
          # The middle node number obtained by looping getMidNodeNum(), starting from 0
          for idx in range(0, midNodeNum):
            nodeName = i+"-"+j
            # Record the new intermediate node name
            Mid.append(nodeName+"-"+str(idx))
            # Count the starting and ending names to hasMidNode
            hasMidNode.append(nodeName)

            # If it is the first intermediate node:
            # 1. It can directly connect to the starting point (back and forth);
            # 2. It can connect to itself (self-looping)
            if idx == 0:
              Z[(i,nodeName+"-"+str(idx))] = 1
              Z[(nodeName+"-"+str(idx),i)] = 1
              Z[(nodeName+"-"+str(idx),nodeName+"-"+str(idx))] = 1
            
            # If it is the last intermediate node:
            # 1. It can directly connect to the ending point (back and forth);
            # 2. It can connect to itself (self-looping)
            if idx == midNodeNum-1:
              if idx != 0:
                Z[(nodeName+"-"+str(idx-1),nodeName+"-"+str(idx))] = 1
                Z[(nodeName+"-"+str(idx),nodeName+"-"+str(idx-1))] = 1
              Z[(nodeName+"-"+str(idx),j)] = 1
              Z[(j,nodeName+"-"+str(idx))] = 1
              Z[(nodeName+"-"+str(idx),nodeName+"-"+str(idx))] = 1
            
            # If it is any other intermediate node: 
            # 1. It can connect to the previous intermediate node; 
            # 2. It can connect to itself (self-looping)
            else:
              Z[(nodeName+"-"+str(idx-1),nodeName+"-"+str(idx))] = 1
              Z[(nodeName+"-"+str(idx),nodeName+"-"+str(idx-1))] = 1
              Z[(nodeName+"-"+str(idx),nodeName+"-"+str(idx))] = 1


tranMidPoints = pd.DataFrame({'Name':Mid})

allNode = list(coordMap.keys()) + Mid
z = pd.DataFrame(index=allNode,columns=allNode)

for i in allNode:
  for j in allNode:
    if (i,j) in Z:
      z[i][j] = Z[i,j]
    else:
      z[i][j] = 0

tranMidPoints.to_csv('./datasets/tranMidPoints.csv')
z.to_csv('./datasets/connections.csv')
